In [1]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date,datetime,timedelta
hoje =  pd.to_datetime(date.today() + timedelta(days=25)) 
final_do_mes = pd.to_datetime(datetime(hoje.year,hoje.month+1,1) - timedelta(seconds=1))

Current working directory was changed to: C:\Users\Vinicius\Documents\GitHub\Previsor\App


In [2]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeeloperacaou

In [3]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
list_id_data = ['DthEnvio','IdeUsinaOutorga']
list_id_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Informações de Usinas

## Carrega banco de dados

In [4]:
cols = ["IdeUsinaOutorga",'NomUsina','CodCegFormatado','DatMonitoramento',"DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado",'DscJustificativaPrevisao']
df_usina = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]
df_usina = df_usina[df_usina.IdcUsinaMonitorada == "Sim"].reset_index(drop=True)

In [5]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]
df_usina = pd.merge(df_usina,monitoramentoleilao,how="left",on="IdeUsinaOutorga")

In [6]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','CodCeg','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma = rapeel_cronograma.loc[rapeel_cronograma.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
df_usina = pd.merge(df_usina,rapeel_cronograma,how="left",on="IdeUsinaOutorga")

In [7]:
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)
mercado = mercado.loc[mercado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()]
del mercado['DthEnvio']
df_usina = pd.merge(df_usina,mercado,on="IdeUsinaOutorga",how="left")

In [8]:
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del ambiental_datas['DthEnvio']
df_usina = pd.merge(df_usina,ambiental_datas,on="IdeUsinaOutorga",how="left")

In [9]:
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del acesso_contratos['DthEnvio']
df_usina = pd.merge(df_usina,acesso_contratos,on="IdeUsinaOutorga",how="left")

##  Cria colunas com definições e regras

In [10]:
df_usina["classe"] = df_usina.CodCegFormatado.str.slice(4,6)
df_usina["CC_real"] = pd.to_datetime(pd.NA)
df_usina.loc[df_usina.classe == "PH","CC_real"] = df_usina.DatRealizacaoIV
df_usina.loc[df_usina.classe == "CV","CC_real"] = df_usina.DatRealizacaoV
df_usina["ME_real"] = df_usina.DatRealizacaoX
df_usina.loc[df_usina.classe == "CV","ME_real"] = df_usina.DatRealizacaoVI

In [11]:
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

df_usina['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    df_usina.loc[df_usina.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = df_usina[validade]
df_usina[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
1752,50017,NaT,NaT,2023-08-05,2023-08-05
2110,52368,NaT,NaT,2023-07-09,2023-07-09
1012,46736,NaT,2031-08-20,2031-08-20,2031-08-20
1152,47302,2031-05-10,2031-05-10,2031-05-10,2031-05-10
2092,52307,NaT,NaT,2026-10-20,2026-10-20


In [12]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao

df_usina['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    df_usina.loc[df_usina[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

df_usina.loc[(df_usina.CondicaoAmbiental.isna()) & (df_usina.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
df_usina.loc[(df_usina.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


df_usina[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(df_usina[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

df_usina['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

df_usina.loc[
    ((df_usina.NomSitContratoCCD ==  nsa)&
    (df_usina.NomSitContratoCCT == nsa) &
    df_usina.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

df_usina.loc[
(    ((df_usina.NomSitContratoCCD ==  valido) |
    (df_usina.NomSitContratoCCT == valido)) &
    df_usina.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

df_usina.loc[
    (((df_usina.NomSitContratoCCD ==  vencido) |
    (df_usina.NomSitContratoCCT == vencido) |
    (df_usina.NomSitContratoCCD == nass) |
    (df_usina.NomSitContratoCCT == nass )) &
    df_usina.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(df_usina[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


df_usina['CondicaoUso'] = "Não informado"

df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  nsa)&
    (df_usina.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  valido) |
    (df_usina.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

df_usina.loc[
    (((df_usina.NomSituacaoContratoCUSD ==  vencido) |
    (df_usina.NomSitContratoCUST == vencido) |
    (df_usina.NomSituacaoContratoCUSD == nass) |
    (df_usina.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(df_usina[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


df_usina["PPA"] = "Ambos"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(df_usina[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2026-03-09,2002-12-11,LI,2026-03-09
1,NaT,NaT,NaT,Sem LP,NaT
2,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
3,NaT,NaT,NaT,Não Informado,NaT
4,NaT,2009-03-10,1999-01-09,LI,2009-03-10
...,...,...,...,...,...
2441,NaT,NaT,NaT,Não Informado,NaT
2442,NaT,NaT,NaT,Não Informado,NaT
2443,NaT,NaT,NaT,Não Informado,NaT
2444,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
2204,54131,Não se Aplica,Não Assinado,2023-01-05 17:41:31.830,Sem Conexão
603,38297,Não se Aplica,Válido,2023-01-10 12:04:29.677,OK
113,32401,Não se Aplica,Não Assinado,2023-01-02 08:40:49.157,Sem Conexão
405,37492,Não se Aplica,Válido,2023-01-03 18:52:55.747,OK
523,37927,NaN,NaN,NaT,Não informado
1002,46721,Não se Aplica,Válido,2023-01-05 18:26:52.690,OK
1286,47478,Não se Aplica,Não Assinado,2023-01-02 19:49:58.607,Sem Conexão
1702,49891,Não se Aplica,Não Assinado,2023-01-02 10:45:33.570,Sem Conexão
2085,52278,NaN,NaN,NaT,Não informado
2404,57211,Não se Aplica,Não Assinado,2023-01-03 09:36:13.790,Sem Conexão


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
1517,49435,Não se Aplica,Válido,Sem Conexão
2211,54139,Não se Aplica,Não Assinado,Sem Conexão
1660,49814,Não Assinado,Não se Aplica,Verificar
2066,52133,Não se Aplica,Não Assinado,Sem Conexão
2134,53642,Não se Aplica,Não se Aplica,Verificar
1776,50113,Não se Aplica,Não Assinado,Sem Conexão
287,35215,Não se Aplica,Válido,OK
1901,50949,NaN,NaN,Não informado
272,35080,Não Assinado,Não se Aplica,Sem Conexão
2343,55340,Não se Aplica,Não Assinado,Sem Conexão


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
1280,47472,Fora do ACR,NaT,Nenhum
207,34188,Fora do ACR,2020-03-03,ACL
456,37643,Fora do ACR,NaT,Nenhum
1654,49709,Fora do ACR,NaT,Nenhum
970,46539,Fora do ACR,NaT,Nenhum
2242,54621,Fora do ACR,NaT,Nenhum
499,37820,ACR,NaT,ACR
1334,48598,Fora do ACR,NaT,Nenhum
2168,53870,Fora do ACR,NaT,Nenhum
1169,47322,Fora do ACR,NaT,Nenhum


In [13]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

In [14]:
slp = "Sem LP"
sim = "Sim"
df_usina['criterio_novo'] = np.select(
    [
        (df_usina.IdcSemPrevisao == sim),
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        (df_usina.CondicaoAmbiental == "Sem LP"),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(df_usina[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo
631,40580,Não Iniciada,ACR,LI,OK,1
1571,49507,Não Iniciada,Nenhum,LO,OK,2
857,44873,Não Iniciada,Nenhum,LP,Verificar,5
1869,50842,Não Iniciada,Nenhum,LO,OK,2
1684,49838,Não Iniciada,Nenhum,LP,OK,4


In [15]:
dict_marcos_homologar = {
    'DatInicioObraRealizado' : 'IO_real',
    'DatConcretagemRealizado' : 'CC_real',
    'DatMontagemRealizado' : 'ME_real',
    'DatEnchimentoRealizado' : 'Ench_Real',
    'DatSisTransmissaoRealizado' : 'IOTrans_Real',
    'DatCanteiroObraRealizado' : 'canteiroReal',
    'DatComissionamentoUGRealizado' : 'Comiss_Real',
    'DatConclusaoSisTransRealizado' : 'Conc_Trans_REAL',
    'DatDesvioRioRealizado' : 'DesvRio_real',
    'DatConclusaoTorresRealizado' : 'ME_Real_conc_eol'}
    
df_usina['Homologar_Marcos'] = False
df_usina['Dsc_Marcos_a_Homologar'] = ''


for marco_monitoramento, marco_rapeel in dict_marcos_homologar.items():
    mask = (df_usina[marco_monitoramento].isna()) & (df_usina[marco_rapeel].notna())
    df_usina['Homologar_Marcos'] |= mask
    df_usina.loc[mask,'Dsc_Marcos_a_Homologar'] += f'{marco_monitoramento}, '

df_usina.Dsc_Marcos_a_Homologar = df_usina.Dsc_Marcos_a_Homologar.str.slice(0,-2)

In [16]:
"""df_usina['Homologar_Marcos'] =  np.select(
    [((df_usina['DatInicioObraRealizado'].isna()) & (df_usina['IO_real'].notna())) |
     ((df_usina['DatConcretagemRealizado'].isna()) & (df_usina['CC_real'].notna())) | 
     ((df_usina['DatMontagemRealizado'].isna()) & (df_usina['ME_real'].notna())) | 
     ((df_usina['DatEnchimentoRealizado'].isna()) & (df_usina['Ench_Real'].notna())) | 
     ((df_usina['DatSisTransmissaoRealizado'].isna()) & (df_usina['IOTrans_Real'].notna())) |
      ((df_usina['DatCanteiroObraRealizado'].isna()) & (df_usina['canteiroReal'].notna())) |
     ((df_usina['DatComissionamentoUGRealizado'].isna()) & (df_usina['Comiss_Real'].notna())) | 
     ((df_usina['DatConclusaoSisTransRealizado'].isna()) & (df_usina['Conc_Trans_REAL'].notna())) | 
     ((df_usina['DatDesvioRioRealizado'].isna()) & (df_usina['DesvRio_real'].notna())) |
     ((df_usina['DatConclusaoTorresRealizado'].isna()) & (df_usina['ME_Real_conc_eol'].notna()))],

    [True],
    default= False
)
"""


df_usina['Revisar_IO'] =  np.select(
 [(df_usina.DatPrevisaoIniciobra < (hoje + pd.Timedelta(15,"D"))) |
 (df_usina.prev_IO > df_usina.DatPrevisaoIniciobra)],
 [True],
 default = False)

In [17]:
usinas_selecionadas = read_file("./usinas_selecionadas.txt").split("\n")
usinas_selecionadas = [int(num) for num in usinas_selecionadas]
df_usina["Usina_Selecionada"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(usinas_selecionadas),'Usina_Selecionada'] = True

df_usina['Paralisada'] = False
df_usina.loc[df_usina.IdcObraParalisada=="Sim",'Paralisada'] = True

df_usina['Sem_Previsao'] = False
df_usina.loc[df_usina.IdcSemPrevisao=="Sim",'Sem_Previsao'] = True

# Informações de UGs

## Carrega banco de dados

In [18]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga",'NumUgUsina',"DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada",'DatLiberOpComerRealizado']
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols].rename(columns={'NumUgUsina':'NumOperacaoUg'})

In [19]:
# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ug_rapeel.loc[ug_rapeel.groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax()]

In [20]:
list_cols_used =   ['IdeUsinaOutorga','IdcObraParalisada','IdcSemPrevisao','criterio_novo','DscJustificativaPrevisao']
df_ug = pd.merge(df_usina[list_cols_used],monitoramentoug,on="IdeUsinaOutorga",how='left')
df_ug = pd.merge(df_ug,ug_rapeel,on=list_id_ug,how='left')
df_ug = df_ug.loc[df_ug.DatLiberOpComerRealizado.isna()].reset_index(drop=True)

In [21]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100

In [22]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm','NumUgUsina':'NumOperacaoUg'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento','DatInicioObraOutorgado','DatPrevisaoIniciobra']:
    if col in calculo_previsao.columns:
        calculo_previsao.drop([col],inplace=True,axis=1)

In [23]:
df_ug = pd.merge(df_ug,calculo_previsao,on=list_id_ug,how="left")

##  Cria colunas com definições e regras

In [24]:
################################ criterionovopmo ################################
list_condicoes = [
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 8)),
    ((df_ug['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

df_ug['criterionovopmo'] = np.select(list_condicoes,list_values)


################################ regranovapmo ################################

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT"),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)


df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].isna()),'regranovapmo'] = df_ug['DatPrevistaComercial']


df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].notna()),'regranovapmo'] = df_ug['Previsao_OC']

df_ug.loc[(df_ug['criterio_novo']== 1) | (df_ug['criterio_novo']== 2) ,'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
df_ug["Dummy"] = hoje + pd.Timedelta(5*365,'D')

df_ug.loc[((df_ug['criterio_novo'] == 3) | (df_ug['criterio_novo'] == 4) | (df_ug['criterio_novo'] == 5) ),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 6,'regranovapmo'] = pd.NA

df_ug["Dummy"] = hoje + pd.Timedelta(6*365,'D')
df_ug.loc[df_ug['criterio_novo']== 7,'regranovapmo'] =  df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 8,'regranovapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[(df_ug['criterio_novo'] == 9),'regranovapmo'] = pd.NA


In [25]:
serie_usinas_previsao_OC_passado = df_ug[(df_ug.Previsao_OC < final_do_mes) | (df_ug.Previsao_OC.isna())].IdeUsinaOutorga.drop_duplicates()
serie_usinas_previsao_em_teste = df_ug[df_ug.criterionovopmo.isin([0.2,0.3])].IdeUsinaOutorga.drop_duplicates()

serie_flagOPTeste30dias = df_ug[df_ug.flagOPTeste30dias == 1].IdeUsinaOutorga.drop_duplicates()
df_usina["flagOPTeste30dias"] = False

df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

In [26]:
df_usina["Prev_OC_passado"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_OC_passado),'Prev_OC_passado'] = True

df_usina["Em_teste"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_em_teste),'Em_teste'] = True

df_usina["Sem_Monitoramento"] = False
df_usina.loc[df_usina.DatMonitoramento < (hoje - pd.Timedelta(4*30, unit="D")),'Sem_Monitoramento'] = True

In [27]:
df_usina["Manual"] = False
df_usina.loc[(df_usina.Paralisada | df_usina.Sem_Previsao | 
    df_usina.Usina_Selecionada| df_usina.Prev_OC_passado | df_usina.Em_teste | 
    (df_usina.DscJustificativaPrevisao == 'Situação das obras de conexão e linha de transmissão associada.')),
    'Manual'] = True

In [28]:
df_usina['Caso I'] = df_usina['Caso II-a'] = df_usina['Caso II-b'] = df_usina['Caso III'] = False
list_casos = ['Caso I','Caso II-a','Caso II-b','Caso III']

In [29]:
df_usina['Caso I'] = (~df_usina.Manual) & (~df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos) #& df_usina.Sem_Monitoramento

df_usina['Caso II-a'] = (~df_usina.Manual) & (df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos)

df_usina['Caso II-b'] = (~df_usina.Manual)  & ((df_usina.Homologar_Marcos) | (df_usina.Homologar_Marcos & df_usina.Revisar_IO))


df_usina['Caso III'] = (
    (df_usina.Manual & df_usina.Sem_Monitoramento ) | 
    (df_usina.Manual & df_usina.Homologar_Marcos ) |
    (df_usina.Manual & df_usina.Revisar_IO ) |
    df_usina.Em_teste )

## Análise de casos BIU

In [30]:
list_criterios_BIU = ['Manual', 'Revisar_IO','Homologar_Marcos', 'Em_teste','Sem_Monitoramento']
casos = df_usina[list_criterios_BIU +  list_casos].value_counts().reset_index().sort_values(by='Caso III')
casos

,Manual,Revisar_IO,Homologar_Marcos,Em_teste,Sem_Monitoramento,Caso I,Caso II-a,Caso II-b,Caso III,0
0,False,False,False,False,False,True,False,False,False,2024
1,True,False,False,False,False,False,False,False,False,129
2,False,False,True,False,False,False,False,True,False,82
3,False,True,False,False,False,False,True,False,False,78
6,False,True,True,False,False,False,False,True,False,19
8,False,False,False,False,True,True,False,False,False,1
4,True,False,False,True,False,False,False,False,True,64
5,True,False,False,False,True,False,False,False,True,40
7,True,False,True,False,False,False,False,False,True,8
9,True,False,True,True,False,False,False,False,True,1


In [31]:
mask_casos_selecionados = casos[list_casos].any(axis=1)
casos[mask_casos_selecionados]

,Manual,Revisar_IO,Homologar_Marcos,Em_teste,Sem_Monitoramento,Caso I,Caso II-a,Caso II-b,Caso III,0
0,False,False,False,False,False,True,False,False,False,2024
2,False,False,True,False,False,False,False,True,False,82
3,False,True,False,False,False,False,True,False,False,78
6,False,True,True,False,False,False,False,True,False,19
8,False,False,False,False,True,True,False,False,False,1
4,True,False,False,True,False,False,False,False,True,64
5,True,False,False,False,True,False,False,False,True,40
7,True,False,True,False,False,False,False,False,True,8
9,True,False,True,True,False,False,False,False,True,1


In [32]:
list_criterios = ['criterio_novo',"criterionovopmo"]

df_usina_criterio = df_ug.loc[df_ug.groupby("IdeUsinaOutorga").criterionovopmo.idxmax()][["IdeUsinaOutorga","FASE",'flagOPTeste30dias'] + list_criterios]

In [33]:
################################ dscjustificativaregranova ################################
list_condicoes = [
    (df_usina_criterio['criterionovopmo'] == 9),
    (df_usina_criterio['criterionovopmo'] == 8),
    (df_usina_criterio['criterionovopmo'] == 0.2),
    (df_usina_criterio['criterionovopmo'] == 0.1) | (df_usina_criterio['criterionovopmo'] == 0.3),
    (df_usina_criterio['criterionovopmo'] == 1),
    df_usina_criterio['criterionovopmo']==2,
    df_usina_criterio['criterionovopmo']==3,
    (df_usina_criterio.criterionovopmo== 4),
    (df_usina_criterio.criterionovopmo== 5),
    (df_usina_criterio.criterionovopmo== 6),
    (df_usina_criterio.criterionovopmo== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

df_usina_criterio['dscjustificativaregranova'] = np.select(list_condicoes,list_values)



################################ dsccriterionovo ################################
list_condicoes = [
    ((df_usina_criterio['criterio_novo'] == 9)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] != "OT")),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT") & (df_usina_criterio['flagOPTeste30dias'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT")),
    ((df_usina_criterio['criterio_novo'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 2)),
    ((df_usina_criterio['criterio_novo'] == 3)),
    ((df_usina_criterio['criterio_novo'] == 4)),
    ((df_usina_criterio['criterio_novo'] == 5)),
    ((df_usina_criterio['criterio_novo'] == 7)),
    ((df_usina_criterio['criterio_novo'] == 6)),
    ((df_usina_criterio['criterio_novo'] == 8))
]

list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras = Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL  = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas = Previsão OC maior entre data RAPEEL e data compromisso"
]
df_usina_criterio['dsccriterionovo'] = np.select(list_condicoes,list_values)
list_justificativas = ['dsccriterionovo','dscjustificativaregranova']

In [34]:
df_usina = pd.merge(df_usina,
df_usina_criterio[['IdeUsinaOutorga','criterionovopmo'] + list_justificativas],
on = "IdeUsinaOutorga", how='left')

In [35]:
#df_usina[df_usina.Homologar_Marcos & (~ df_usina.Sem_Monitoramento)][['IdeUsinaOutorga','NomUsina','Homologar_Marcos',"Sem_Monitoramento",'DatMonitoramento','DthEnvio','Dsc_Marcos_a_Homologar']].to_excel("Usinas_sem_fotos.xlsx",index=False)

# Testes

## Junta dados do SAS e Python

In [36]:
criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga").rename(columns={'NumUgUsina':'NumOperacaoUg'})[["IdeUsinaOutorga","NumOperacaoUg",'CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo','CalculoPrevisorOC_mx',"FASE"]]

criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
df_ug["PY"] = True


#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x003D_','=')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002C_', ',')

In [37]:
criterios_da_previsao_sas_gb = criterios_da_previsao_sas.groupby("IdeUsinaOutorga")[['CriterioNovoPMO','DscCriterioNovo','DscJustificativaRegranova']].nunique()
usinas_dois_criterios = criterios_da_previsao_sas_gb[criterios_da_previsao_sas_gb.CriterioNovoPMO>1].index

In [38]:
criterios_da_previsao_sas_usina = criterios_da_previsao_sas[['IdeUsinaOutorga','CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo']].drop_duplicates()

In [39]:
merge_usina  = pd.merge(df_usina_criterio[['IdeUsinaOutorga','criterionovopmo','dscjustificativaregranova','dsccriterionovo']],
criterios_da_previsao_sas_usina,on=['IdeUsinaOutorga'],suffixes=('_py','_sas'),how='outer')

In [40]:
criterios_da_previsao_sas_ug = criterios_da_previsao_sas[list_id_ug + ['CalculoPrevisorOC_mx','FASE']]

In [41]:
merge_ug = pd.merge(df_ug[list_id_ug + ['FASE','Previsao_OC']],criterios_da_previsao_sas_ug,on=['IdeUsinaOutorga','NumOperacaoUg'],suffixes=('_py','_sas'),how='outer')
merge_ug['OC_diff'] = merge_ug.Previsao_OC - merge_ug.CalculoPrevisorOC_mx

## Teste previsão

In [42]:
merge_ug.sort_values(by='OC_diff',ascending=False).head(20)

,IdeUsinaOutorga,NumOperacaoUg,FASE_py,Previsao_OC,CalculoPrevisorOC_mx,FASE_sas,OC_diff
119,30114,1,OUT,2024-08-13,2021-12-01,OUT,986 days
138,30414,1,OUT,2025-03-18,2022-09-08,OUT,922 days
120,30114,2,OUT,2024-08-27,2022-05-19,OUT,831 days
144,30612,1,OUT,2025-03-29,2023-01-23,OUT,796 days
118,30114,3,OUT,2024-09-02,2022-08-11,OUT,753 days
11617,37731,2,OUT,2024-09-25,2023-02-24,OUT,579 days
11618,37731,3,OUT,2024-09-25,2023-02-24,OUT,579 days
11619,37731,1,OUT,2024-09-25,2023-02-24,OUT,579 days
11613,37730,1,OUT,2024-10-01,2023-05-16,OUT,504 days
11615,37730,2,OUT,2024-10-01,2023-05-16,OUT,504 days


## Teste critérios

### FASE ok

In [43]:
merge_ug[merge_ug.FASE_py != merge_ug.FASE_sas]

,IdeUsinaOutorga,NumOperacaoUg,FASE_py,Previsao_OC,CalculoPrevisorOC_mx,FASE_sas,OC_diff


### criterionovopmo OK

In [44]:
merge_usina[(merge_usina.CriterioNovoPMO != merge_usina.criterionovopmo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
6,1928,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
172,33547,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
513,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
650,40626,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...


In [45]:
merge_usina[(merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))].sample(6)

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
579,38106,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
781,44363,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
376,37101,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
177,33549,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
580,38106,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
140,32580,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...


### dscjustificativaregranova OK

In [46]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]


,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
6,1928,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
172,33547,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
513,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
650,40626,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...


In [47]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova)]

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
6,1928,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
100,32102,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
133,32568,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
139,32580,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
172,33547,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
174,33548,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
177,33549,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
375,37101,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
513,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
566,38081,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...


### dsccriterionovo OK

In [48]:
merge_usina[(merge_usina.dsccriterionovo != merge_usina.DscCriterioNovo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
6,1928,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
172,33547,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
513,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
650,40626,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...


# BIU

In [49]:
import os
skate_downloads_folder_name = "SKATE_Downloads"
root_path = os.path.join(get_standard_folder_path("Documents"), "Previsor")
download_path = os.path.join(root_path, skate_downloads_folder_name)



'C:\\Users\\Vinicius\\Documents\\Previsor\\SKATE_Downloads\\2023_01_27'

In [51]:
log_path = f'{directory}/log.pickle'

In [52]:
log = load_pickle(log_path)

In [53]:
for col in log:
    log[col] = datetime(2023,1,17,0)

In [54]:
save_pickle(log,log_path)

In [55]:
df_usina

,IdeUsinaOutorga,NomUsina,CodCegFormatado,DatMonitoramento,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra,IdcObraParalisada,IdcUsinaMonitorada,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra,IdcSemPrevisao,DatComissionamentoUGRealizado,SigTipoGeracao,DscComercializacaoEnergia,DatInicioObraRealizado,DatMontagemRealizado,DscJustificativaPrevisao,CodLeilao,DatInicioSuprimento,CodCeg,DthEnvio,canteiroReal,prev_IO,IO_real,DesvRio_real,Ench_Real,IOTrans_Real,Conc_Trans_REAL,Comiss_Real,ME_Real_conc_eol,Conc_Combust_Real,DatRealizacaoVI,DatRealizacaoX,DatRealizacaoV,DatRealizacaoIV,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT,classe,CC_real,ME_real,ValidadeAmbiental,CondicaoAmbiental,CondicaoConexao,CondicaoUso,PPA,criterio_novo,Homologar_Marcos,Dsc_Marcos_a_Homologar,Revisar_IO,Usina_Selecionada,Paralisada,Sem_Previsao,flagOPTeste30dias,Prev_OC_passado,Em_teste,Sem_Monitoramento,Manual,Caso I,Caso II-a,Caso II-b,Caso III,criterionovopmo,dsccriterionovo,dscjustificativaregranova
0,324,Bugres,UHE.PH.RS.000324-7,2022-10-13 11:01:56.560,NaT,NaT,NaT,NaT,NaT,Não,Sim,2015-03-31,NaT,NaT,2014-02-28,2014-12-31,NaT,Não Iniciada,Sim,NaT,UHE,Fora do ACR,NaT,NaT,Inviabilidade da implantação da usina,NaN,NaT,UHEPHRS000324-7,2023-01-02 08:34:07.347,NaT,2025-02-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2026-03-09,2002-12-11,NaT,Não se Aplica,Não se Aplica,Válido,Válido,PH,NaT,NaT,2026-03-09,LI,OK,OK,Nenhum,9,False,,False,False,False,True,False,False,False,True,True,False,False,False,True,9.0,Usina Viabilidade Baixa análise da fiscalização,Analisar justificativa: Revogação da outorga e...
1,601,Caldeirões,PCH.PH.MG.000601-7,2022-10-13 11:03:48.340,NaT,NaT,NaT,NaT,NaT,Não,Sim,2001-02-15,NaT,NaT,2001-01-30,NaT,NaT,Não Iniciada,Sim,NaT,PCH,Fora do ACR,NaT,NaT,Nenhuma licença ambiental válida,NaN,NaT,PCHPHMG000601-7,2023-01-03 19:22:43.140,NaT,2026-05-29,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Não Assinado,Não se Aplica,Não Assinado,Não se Aplica,PH,NaT,NaT,NaT,Sem LP,Sem Conexão,Sem Uso,Nenhum,9,False,,False,False,False,True,False,False,False,True,True,False,False,False,True,9.0,Usina Viabilidade Baixa análise da fiscalização,Analisar justificativa: Revogação da outorga e...
2,659,Capivari,PCH.PH.SC.000659-9,2022-12-13 17:04:26.787,2012-12-31,2013-10-29,2017-07-24,2016-09-17,NaT,Sim,Sim,2005-02-28,2014-01-30,NaT,2002-04-30,2004-08-31,2013-10-01,Paralisada,Sim,NaT,PCH,ACR,2012-12-31,2014-10-29,Paralisação de obras,01/2016,2021-01-01,PCHPHSC000659-9,2023-01-06 09:37:52.703,NaT,2023-02-10,NaT,2016-07-30,2017-07-24,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2028-06-28,2000-03-29,2026-02-22,Válido,Válido,Válido,Válido,PH,NaT,NaT,2026-02-22,LO,OK,OK,ACR,9,False,,False,False,True,True,False,False,False,False,True,False,False,False,False,9.0,Usina Viabilidade Baixa análise da fiscalização,Analisar justificativa: Revogação da outorga e...
3,1195,Itumirim,UHE.PH.GO.001195-9,2022-10-13 11:06:18.107,NaT,NaT,NaT,NaT,NaT,Não,Sim,NaT,NaT,NaT,2002-03-31,2002-03-31,NaT,Não Iniciada,Sim,NaT,UHE,Fora do ACR,NaT,NaT,Revogação da outorga em avaliação,NaN,NaT,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,PH,NaT,NaT,NaT,Não Informado,Não informado,Não informado,Nenhum,9,False,,False,False,False,True,False,True,False,True,True,False,False,False,True,9.0,Usina Viabilidade Baixa análise da fiscalização,Analisar justificativa: Revogação da outorga e...
4,1358,Machadinho I,PCH.PH.RO.001358-7,2022-11-10 16:22:08.193,NaT,NaT,NaT,NaT,NaT,Não,Sim,2015-05-30,NaT,NaT,2014-08-30,2015-03-30,NaT,Não Iniciada,Sim,NaT,PCH,Fora do ACR,NaT,NaT,Revogação da outorga em avaliação,NaN,NaT,PCHPHRO001358-7,2022-07-05 18:10:51.733,NaT,2023-02

In [56]:
ug_com_previsao = df_ug.loc[df_ug.DatPrevistaComercial.notna()][list_id_ug + ['DatPrevistaComercial']]
ug_com_previsao = ug_com_previsao.loc[ug_com_previsao.groupby('IdeUsinaOutorga').DatPrevistaComercial.idxmax()][['IdeUsinaOutorga','DatPrevistaComercial']].rename(
    columns = {'DatPrevistaComercial': 'DatPrevistaComercial_MaxUsina'})
df_ug = pd.merge(df_ug,ug_com_previsao,on='IdeUsinaOutorga')